# Taller Integrador – Sistema Experto de Eficiencia Energética

Este notebook muestra una versión sencilla y funcional del taller, con catálogos simples (listas/diccionarios) y reglas mínimas para pasar las pruebas del autograder.

In [ ]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [ ]:
from experta import *

In [ ]:
# 1. Catálogos simples (listas y diccionarios)


dispositivos = [
    {"tipo": "aire_acondicionado", "baseline": 7.2, "umbral": 9.0},
    {"tipo": "iluminacion", "baseline": 0.36, "umbral": 0.45},
    {"tipo": "nevera", "baseline": 2.88, "umbral": 3.6},
]

habitos = [
    {"codigo": "ac_puertas_abiertas", "activo": True, "dispositivo": "aire_acondicionado"},
    {"codigo": "luces_encendidas_sin_uso", "activo": True, "dispositivo": "iluminacion"},
]

# ************************************************
# 2. Definición de hechos


class Dispositivo(Fact): pass
class Habito(Fact): pass
class Medicion(Fact): pass
class Hipotesis(Fact): pass
class Accion(Fact): pass

#************************************************
# 3. Motor de inferencia


class SistemaEnergia(KnowledgeEngine):

    @Rule(Dispositivo(tipo='aire_acondicionado'),
          Medicion(dispositivo='aire_acondicionado', kwh_dia=MATCH.k),
          TEST(lambda k: k > 9.0))
    def ac_sobreconsumo(self, k):
        print(f"ACTIVADA: ac_sobreconsumo | kwh={k}")
        self.declare(Hipotesis(motivo='sobreconsumo_clima', severidad='alta'))
        self.declare(Accion(texto='Cerrar puertas y mantener setpoint 24°C', dispositivo='aire_acondicionado'))

    @Rule(Habito(codigo='ac_puertas_abiertas', activo=True),
          Hipotesis(motivo='sobreconsumo_clima', severidad='alta'))
    def ac_habito_empeora(self):
        print("ACTIVADA: ac_habito_empeora | hábito=ac_puertas_abiertas")
        for fid, fact in list(self.facts.items()):
            if isinstance(fact, Hipotesis):
                self.retract(fid); break
        self.declare(Hipotesis(motivo='sobreconsumo_clima', severidad='critica'))

    @Rule(Dispositivo(tipo='iluminacion'),
          Habito(codigo='luces_encendidas_sin_uso', activo=True),
          Medicion(dispositivo='iluminacion', kwh_dia=MATCH.k),
          TEST(lambda k: k > 0.36))
    def luces_ineficientes(self, k):
        print(f"ACTIVADA: luces_ineficientes | kwh={k}")
        self.declare(Hipotesis(motivo='iluminacion_ineficiente', severidad='media'))
        self.declare(Accion(texto='Apagar luces sin uso o instalar sensores', dispositivo='iluminacion'))

    @Rule(Dispositivo(tipo='nevera'),
          Medicion(dispositivo='nevera', kwh_dia=MATCH.k),
          TEST(lambda k: k > 3.6))
    def nevera_ajuste(self, k):
        print(f"ACTIVADA: nevera_ajuste | kwh={k}")
        self.declare(Hipotesis(motivo='ajuste_temp_nevera', severidad='media'))
        self.declare(Accion(texto='Revisar temperatura (3–5°C)', dispositivo='nevera'))

    @Rule(OR(
        AND(Hipotesis(motivo='sobreconsumo_clima', severidad=MATCH.s),
            Medicion(dispositivo='aire_acondicionado', kwh_dia=MATCH.k1),
            TEST(lambda k1: k1 <= 7.2)),
        AND(Hipotesis(motivo='iluminacion_ineficiente', severidad=MATCH.s),
            Medicion(dispositivo='iluminacion', kwh_dia=MATCH.k2),
            TEST(lambda k2: k2 <= 0.36))
        ))
    def normalizacion(self, s, k1=None, k2=None):
        print("ACTIVADA: normalizacion | consumo <= baseline")
        self.declare(Hipotesis(motivo='consumo_normalizado', severidad='baja'))

# ***************************************************
# 4. Declaración base


def declarar_base(engine):
    engine.declare(Dispositivo(tipo='aire_acondicionado'))
    engine.declare(Dispositivo(tipo='iluminacion'))
    engine.declare(Dispositivo(tipo='nevera'))
    engine.declare(Habito(codigo='ac_puertas_abiertas', activo=True))
    engine.declare(Habito(codigo='luces_encendidas_sin_uso', activo=True))

# *************************
# 5. Escenarios


def resumen_facts(engine):
    hips = [dict(f) for f in engine.facts.values() if isinstance(f, Hipotesis)]
    accs = [dict(f) for f in engine.facts.values() if isinstance(f, Accion)]
    return {"hipotesis": hips, "acciones": accs}

def _run_A():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='aire_acondicionado', kwh_dia=9.8))
    e.run(); return resumen_facts(e)

def _run_B():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='iluminacion', kwh_dia=0.5))
    e.run(); return resumen_facts(e)

def _run_C():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='iluminacion', kwh_dia=0.5))
    e.run()
    e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='iluminacion', kwh_dia=0.35))
    e.run(); return resumen_facts(e)

RESULTADOS = {"A": _run_A(), "B": _run_B(), "C": _run_C()}
RESULTADOS